In [1]:
import sys
sys.path.append('../')
import jax.numpy as jnp


/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/jax/_src/lib/__init__.py:32: UserWarning: JAX on Mac ARM machines is experimental and minimally tested. Please see https://github.com/google/jax/issues/5501 in the event of problems.
  warnings.warn("JAX on Mac ARM machines is experimental and minimally tested. "


In [2]:
import src.models.rnn as rnn
from src.utils.utils import *
import jax

In [3]:
trf=rnn.rnn_transform(rnn.MultiplicativeRNN,10,4,32)

In [4]:
key=jax.random.PRNGKey(0)
sample_o=jax.random.normal(key,[20,10])
sample_a=jax.random.normal(key,[20,4])
h_t=rnn.MultiplicativeRNN.initial_state(10,4,32,1)
rnn_params=trf.init(key,(sample_o[0],sample_a[0]),h_t)

In [5]:
for i in range(12):
    out,h_t=(trf.apply)(rnn_params,(sample_o[i],sample_a[i]),h_t)

In [6]:
def hvp(rnn_params,inputs,last_state,vector):
    def jvp(rnn_params,inputs,h_t,weights):
        jacobians=jax.jacrev(trf.apply)(rnn_params,inputs,h_t)[0]
        jvp=jax.tree_multimap(lambda x, y: jnp.tensordot(x,y,axes=y.ndim), jacobians, weights)
        jvp=jnp.stack(jax.tree_util.tree_flatten(jvp)[0],axis=0).sum(axis=0)
        return jvp
    hvp_fn=jax.jacrev(jvp)
    return hvp_fn(rnn_params,inputs,last_state,vector)

In [7]:
hvp=jax.jit(hvp)

In [8]:
import time

In [100]:
start=time.time()
h=hvp(rnn_params,(sample_o[0],sample_a[0]),h_t,rnn_params)
time.time()-start

0.01276087760925293

In [101]:
h

FlatMap({
  'multiplicative_rnn': FlatMap({
                          'b': DeviceArray([[[ 9.6794805e-03, -1.1301146e-02,  2.8433993e-02,
                                              -1.3579360e-02],
                                             [-0.0000000e+00,  0.0000000e+00, -0.0000000e+00,
                                               0.0000000e+00],
                                             [-0.0000000e+00,  0.0000000e+00, -0.0000000e+00,
                                               0.0000000e+00],
                                             ...,
                                             [-0.0000000e+00,  0.0000000e+00, -0.0000000e+00,
                                               0.0000000e+00],
                                             [-0.0000000e+00,  0.0000000e+00, -0.0000000e+00,
                                               0.0000000e+00],
                                             [-0.0000000e+00,  0.0000000e+00, -0.0000000e+00,
                           

### HVP: Pearlmutter's method

In [ ]:
def hvp_pm(rnn_params,inputs,last_state,vector):
    def r_jacob(r,rnn_params,inputs,last_state,vector):
        theta_plus_rw=tree_sum(rnn_params,tree_scalar_multiply(vector,r))
        return jax.jacrev(trf.apply)(theta_plus_rw,inputs,last_state)[0]
    r=jnp.zeros(1)
    #return r_jacob(r,rnn_params,inputs,last_state,vector)
    return jax.jacfwd(r_jacob)(r,rnn_params,inputs,last_state,vector)

In [ ]:
start=time.time()
h=hvp_pm(rnn_params,(sample_o[0],sample_a[0]),h_t,rnn_params)
time.time()-start

In [ ]:
h